In [43]:
import pandas as pd

import statsmodels.formula.api as smf

In [7]:
data = pd.read_stata("../data/deaths.dta")
data.head()

,year,state,legal1820,dtype,agegr,count,pop,age,legal,beertaxa,beerpercap,winepercap,spiritpercap,totpercap,mrate
0,1970,1,0.0,all,15-17 yrs,224,213574,15.978335,0.0,1.373711,0.60,0.09,0.70,1.38,104.881683
1,1971,1,0.0,all,15-17 yrs,241,220026,15.985843,0.0,1.316049,0.66,0.09,0.76,1.52,109.532509
2,1972,1,0.0,all,15-17 yrs,270,224877,15.985734,0.0,1.275120,0.74,0.09,0.78,1.61,120.065636
3,1973,1,0.0,all,15-17 yrs,258,227256,15.992462,0.0,1.200450,0.79,0.10,0.79,1.69,113.528358
4,1974,1,0.0,all,15-17 yrs,224,229025,16.000776,0.0,1.081136,0.83,0.16,0.81,1.80,97.805916


In [41]:
data = data[
    (data['year']<=1983)
    & (data['agegr']=='18-20 yrs')
    & (data['dtype']=='all')
].dropna()

data.head()

,year,state,legal1820,dtype,agegr,count,pop,age,legal,beertaxa,beerpercap,winepercap,spiritpercap,totpercap,mrate
1382,1970,1,0.0,all,18-20 yrs,292,189770,18.969990,0.0,1.373711,0.60,0.09,0.70,1.38,153.870468
1399,1971,1,0.0,all,18-20 yrs,316,195623,18.972181,0.0,1.316049,0.66,0.09,0.76,1.52,161.535202
1416,1972,1,0.0,all,18-20 yrs,320,200988,18.974701,0.0,1.275120,0.74,0.09,0.78,1.61,159.213486
1433,1973,1,0.0,all,18-20 yrs,291,206550,18.970230,0.0,1.200450,0.79,0.10,0.79,1.69,140.885986
1450,1974,1,0.0,all,18-20 yrs,306,213839,18.980139,0.0,1.081136,0.83,0.16,0.81,1.80,143.098312


# Causal Inference


According to Equation(5.6) in the book, the reduced form of DID equation (with time trend) is as follows:

$Y_{st} = \alpha + \delta_{rDID}LEGAL_{st} + \sum_{k}\beta_{k}STATE_{ks} + \sum_{j}\gamma_{j}YEAR_{jt} + \sum_{k} \theta_{k}(STATE_{ks} \times t) + \epsilon_{st}$

where:
* $LEGAL_{st}$ is the treatment variable
* $YEAR_{jt}$ is the time-fixed effect
* $STATE_{ks}$ is the state-fixed effect
* $STATE_{ks} \times t$ is the time trend

In the followings, we are interested in the causal effect to all-death death rate when "fractional legal" is controlled, which corresponds to the estimate shown in the first row and third columns of table 5.3, with an estiamte of 10.03 and stand error around 4.92

## `DoWhy` Implementation

Not recommended. The [DID API](https://github.com/py-why/dowhy/issues/635) is still under development.

## `statsmodels` Implementation

In [42]:
formula = 'mrate ~ legal + beertaxa + C(state) * year + C(year)'

sm_did_model = smf.ols(
    formula=formula,
    data=data
)

# regressing with clustered standard error
results = sm_did_model.fit(
    cov_type='cluster',
    cov_kwds={'groups': data['state']}
)

print(
    "OLS Regression Results: \n -- Coef: {:.2f} \n --   SE: {:.2f}".format(
        results.params['legal'], results.bse['legal']
    )
)

# print(results.summary())

OLS Regression Results: 
 -- Coef: 10.03 
 --   SE: 4.92
